In [1]:
import torch

In [2]:
use_cuda = True

device = torch.device('cuda' if use_cuda and torch.cuda.is_available() else 'cpu')

In [3]:
device

device(type='cuda')

In [ ]:
# Data

In [1]:
* `get_mnist`
* `build_data_loaders`

SyntaxError: invalid syntax (<ipython-input-1-8b171d0b810b>, line 1)

In [4]:
from data_utils import get_mnist, build_data_loaders

In [5]:
train_dataset, test_dataset = get_mnist(normalize=True)

In [6]:
# Average of average per-pixel mean
train_dataset.data.mean(0).mean().item(), test_dataset.data.mean(0).mean().item()

(-7.328089801639237e-10, 0.002495675580576062)

In [7]:
# Build data loaders
batch_size = 64
train_loader, test_loader = build_data_loaders(train_dataset, test_dataset, batch_size)

In [8]:
for x, y in train_loader:
    print(x.shape, y.shape)
    break

torch.Size([64, 1, 28, 28]) torch.Size([64])


# Model

In [9]:
from net import Net

In [10]:
net = Net().to(device)

# Custom optimizers

In [11]:
from torch.optim import Optimizer

In [12]:
class MyFancyOptimizer(Optimizer):
    
    def __init__(self, params, lr):
        super(self, MyFancyOptimizer).__init__()

# Training

In [13]:
from torch.optim import SGD

In [14]:
opt = SGD(net.parameters(), lr=0.01)

In [15]:
from training import training, accuracy
from torch.nn import CrossEntropyLoss

In [16]:
losses, accs = training(net, train_loader, opt, CrossEntropyLoss(), accuracy, epochs=5, device=device)

Launching training on cuda
epoch 0	avg epoch loss = 2.326	avg epoch acc = 0.07812
epoch 1	avg epoch loss = 2.314	avg epoch acc = 0.09375
epoch 2	avg epoch loss = 2.302	avg epoch acc = 0.1094
epoch 3	avg epoch loss = 2.291	avg epoch acc = 0.125
epoch 4	avg epoch loss = 2.28	avg epoch acc = 0.1406
training took 0.05831 s


# Test

In [17]:
from training import testing

In [18]:
losses, metrics = testing(net, test_loader, CrossEntropyLoss(), accuracy, device)

Avg loss = 2.301611997519329	Avg acc = 0.11236066878980892
